<a href="https://colab.research.google.com/github/timofeiryko/itmo-cpp/blob/reg-poseidon-copy-adelina/cpp_regr_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libxkbfile1 libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  default-jre default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libxkbfile1 libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
0 upgraded, 12 newly installed, 0 to remove and 29 not upgraded.
Need to get 3,720 kB of archives.
After this operation, 12.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 default-jre-headless amd64 2:1.11-72build2 [3,042 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxtst6 amd64 2:1.2.3-1build4 [13.4 kB]
Get:3 http://archive.ubuntu.com

In [2]:
! pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=8c6d0b84c31a3346df02526ac08a5522c5472465106b554981e6cb775a1f053f
  Stored in directory: /root/.cache/pip/wheels/62/f9/aa/687bd54342d2981bc78e22ee9b9bc39f92006e344e7aa1e0ac
Successfully built h2o


In [3]:
import h2o

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpu3mb_w4b
  JVM stdout: /tmp/tmpu3mb_w4b/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpu3mb_w4b/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 24 days
H2O_cluster_name:,H2O_from_python_unknownUser_udj42i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [32]:
import numpy as np
import pandas as pd
import re
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from h2o.automl import H2OAutoML
import json

In [6]:
def convert_to_number(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(' ', '')  # удаляем пробелы
    # обработка значений '<число'
    if val.startswith('<'):
        num = re.findall(r'<(\d+\.?\d*)', val)
        return float(num[0]) if num else np.nan
    # обработка значений с ±
    elif '±' in val:
        nums = re.findall(r'([\d\.]+)±([\d\.]+)', val)
        if nums:
            main, uncertainty = nums[0]
            return float(main)  # берём только среднее (первое число)
        else:
            return np.nan
    # обработка значений с '/'
    elif '/' in val:
        nums = re.findall(r'([\d\.]+)/([\d\.]+)', val)
        if nums:
            num1, num2 = nums[0]
            return (float(num1) + float(num2)) / 2
        else:
            return np.nan
    # пробуем просто преобразовать в число
    else:
        try:
            return float(val)
        except:
            return np.nan

In [12]:
# Load dataset
df = pd.read_csv("for_regr_with_descrip.csv")
df['raw_efficiency'] = df['raw_efficiency'].apply(convert_to_number)

# Load embeddings
blomap_embeddings = np.load("blomap_regr.npy")
fingerprints_embeddings = np.load("fingerprints_regr.npy")
protbert_embeddings = np.load("protbert_regr.npy")

In [13]:
# Select numerical features
selected_features = ["raw_efficiency", "uptake_type",
    "MW", "GRAVY", "pI", "Charge", "Charge_Density", "Aromaticity",
    "Flexibility", "Aliphatic_Index", "Boman_Index", "Hydrophobic_AA",
    "Polar_AA", "Positive_AA", "Negative_AA", "MolWt", "LogP",
    "TPSA", "HBD", "HBA", "RotBonds", "Rings", "Fsp3"
]
X_numerical = df[selected_features].copy()

In [11]:
def emb_analysis(emb):
  embeddings = emb

  # Просмотр формы массива
  print(f"Форма массива: {embeddings.shape}")

  # Если вы хотите увидеть первые несколько эмбеддингов
  print("Первые 5 эмбеддингов:")
  print(embeddings[:5])

for emb in [blomap_embeddings, fingerprints_embeddings, protbert_embeddings]:
  emb_analysis(emb)

Форма массива: (877, 1345)
Первые 5 эмбеддингов:
[[0.62 0.29 0.   ... 0.   0.   0.  ]
 [0.62 0.29 0.   ... 0.   0.   0.  ]
 [0.62 0.29 0.   ... 0.   0.   0.  ]
 [0.62 0.29 0.   ... 0.   0.   0.  ]
 [0.62 0.29 0.   ... 0.   0.   0.  ]]
Форма массива: (877, 2048)
Первые 5 эмбеддингов:
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
Форма массива: (877, 1024)
Первые 5 эмбеддингов:
[[ 0.03056029  0.02434671  0.13633646 ... -0.08684257 -0.1144724
  -0.01299001]
 [ 0.03056029  0.02434671  0.13633646 ... -0.08684257 -0.1144724
  -0.01299001]
 [ 0.03056029  0.02434671  0.13633646 ... -0.08684257 -0.1144724
  -0.01299001]
 [ 0.03056029  0.02434671  0.13633646 ... -0.08684257 -0.1144724
  -0.01299001]
 [ 0.03056029  0.02434671  0.13633646 ... -0.08684257 -0.1144724
  -0.01299001]]


In [14]:
# Фильтруем датафрейм по столбцу 'uptake_type'
X_numerical_filtered = X_numerical[X_numerical['uptake_type'] == 'Mean Fluorescence intensity'].copy()
X_numerical_filtered = X_numerical_filtered[X_numerical_filtered['raw_efficiency'] <= 50000]

# Получаем индексы отфильтрованных молекул
filtered_indices = X_numerical_filtered.index

# Фильтруем массивы эмбеддингов
blomap_embeddings_filtered = blomap_embeddings[filtered_indices]
fingerprints_embeddings_filtered = fingerprints_embeddings[filtered_indices]
protbert_embeddings_filtered = protbert_embeddings[filtered_indices]

In [13]:
def filter_data(df, arrays, cell_line_array=None, is_y=False):
    """
    Фильтрует данные по маске из столбца 'uptake_type' датафрейма df.

    Параметры:
    - df: pandas.DataFrame с исходными данными
    - arrays: список массивов для фильтрации и объединения (например, [X_numerical, blomap_pca])
    - cell_line_array: дополнительный массив (X_cell_line), который может отсутствовать
    - is_y: флаг, что обрабатываем целевую переменную y

    Возвращает:
    - Отфильтрованный массив или датафрейм
    """
    # Создаем булеву маску
    mask = df['uptake_type'].isin(['Mean Fluorescence intensity', 'Fluorescence intensity'])

    if is_y:
        # Для y просто применяем маску к столбцу
        return df['raw_efficiency'].values[mask]
    else:
        # Фильтруем все переданные массивы
        filtered_arrays = [arr[mask] for arr in arrays]

        # Объединяем массивы
        filtered_X = np.hstack(filtered_arrays)

        # Добавляем cell_line, если он не пуст
        if cell_line_array is not None and not cell_line_array.empty:
            filtered_cell_line = cell_line_array.values[mask]
            filtered_X = np.hstack([filtered_X, filtered_cell_line])

        return filtered_X

In [ ]:
'''
# One-hot encoding for cell_line
if "cell_line" in df.columns:
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cell_line_encoded = enc.fit_transform(df[['cell_line']])
    cell_line_feature_names = enc.get_feature_names_out(["cell_line"])
    X_cell_line = pd.DataFrame(cell_line_encoded, columns=cell_line_feature_names)
else:
    X_cell_line = pd.DataFrame()
'''

In [15]:
# Функция для удаления выбросов методом IQR
def remove_outliers(df, target_column):
    """
    Удаляет выбросы из числовых колонок методом межквартильного размаха (IQR).

    Аргументы:
    df — pandas DataFrame с числовыми признаками.
    target_column — название столбца с таргетом (raw_efficiency).

    Возвращает:
    Очищенный DataFrame без выбросов.
    """
    df_clean = df.copy()

    Q1 = df_clean['raw_efficiency'].quantile(0.25)
    Q3 = df_clean['raw_efficiency'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_clean = df_clean[(df_clean['raw_efficiency'] >= lower_bound) & (df_clean['raw_efficiency'] <= upper_bound)]

    return df_clean

# Удаляем выбросы из числовых признаков
X_numerical_filtered_no_outliers = remove_outliers(X_numerical_filtered, 'raw_efficiency')

# Проверяем размер до и после удаления выбросов
print(f"Размер данных ДО удаления выбросов: {X_numerical_filtered.shape}")
print(f"Размер данных ПОСЛЕ удаления выбросов: {X_numerical_filtered_no_outliers.shape}")

Размер данных ДО удаления выбросов: (170, 23)
Размер данных ПОСЛЕ удаления выбросов: (150, 23)


Привет. У меня есть датасет с химическими данными и три файла с эмбеддингами для молекул, которые представлены в датасете. Датасет сейчас находится в датафрейме X_numerical. В датасете также содержатся дескрипторы, полученные с помощью rdkit. Эмбеддинги загружены в массивы numpy с названиями blomap_embeddings, fingerprints_embeddings, protbert_embeddings. Таргетная величина находится в датафрейме X_numerical и называется 'raw_efficiency'. Это непрерывное значение, поэтому модель будет решать задачу регрессии. Я хочу запустить на этих данных automl h2o в google colab. Я уже установила нужные библиотеки, включая automl h2o. Напиши для меня код, который: 1) отфильтрует значения в датасете, в том числе в массивах, так, чтобы у меня остались только молекулы, у которых в датафрейме указано значение 'Mean Fluorescence intensity' в столбце 'uptake_type'; 2) запустит automl h2o четыре раза: в первый раз будут использоваться дескрипторы RDKit из датафрейма, в следующие три - эмбеддинги из одного из массивов. Для каждого запуска должна быть выведена статистика.

In [16]:
# Разделяем фичи и целевую переменную
target = 'raw_efficiency'
rdkit_descriptors = X_numerical_filtered.drop(columns=['uptake_type', target])

In [34]:
# Функция для запуска H2O AutoML
def run_h2o_automl(X, y, run_name, max_runtime_secs=10, saving=False):
    # Преобразуем данные в H2OFrame
    df_h2o = h2o.H2OFrame(pd.concat([X, y], axis=1))

    # Разделяем на тренировочную и тестовую выборки
    train, test = df_h2o.split_frame(ratios=[0.8], seed=42)

    # Запускаем AutoML
    aml = H2OAutoML(max_runtime_secs=max_runtime_secs, seed=42)
    aml.train(x=X.columns.tolist(), y=target, training_frame=train)

    # Выводим лидирующую модель и статистику

    print(f"\n=== {run_name} ===")
    print(aml.leaderboard)
    print("\nЛучший алгоритм:", aml.leader.algo)
    print("\nМетрики на тестовой выборке:")
    test_perf = aml.leader.model_performance(test)
    train_perf = aml.leader.model_performance(train)
    print(test_perf)
    print(f"R2: {test_perf.r2()}")
    print("\nМетрики на тренировочной выборке:")
    print(train_perf)
    print(f"R2: {train_perf.r2()}")

    if saving:

        results = {
            "run_name": run_name,
            "best_model": aml.leader.algo,
            "metrics_test": {
                "MSE": test_perf.mse(),
                "RMSE": test_perf.rmse(),
                "MAE": test_perf.mae(),
                "RMSLE": test_perf.rmsle(),
                "Mean Residual Deviance": test_perf.mean_residual_deviance(),
                "R2": test_perf.r2()
            },
            "metrics_train": {
                "MSE": train_perf.mse(),
                "RMSE": train_perf.rmse(),
                "MAE": train_perf.mae(),
                "RMSLE": train_perf.rmsle(),
                "Mean Residual Deviance": train_perf.mean_residual_deviance(),
                "R2": train_perf.r2()
            }
        }

        # Сохраняем метрики в JSON
        with open(f"{run_name}_metrics.json", "w") as f:
            json.dump(results, f, indent=4)

       # Сохраняем лидерборд в CSV
        leaderboard_df = aml.leaderboard.as_data_frame()
        leaderboard_df.to_csv(f"{run_name}_leaderboard.csv", index=False)

    #    # Сохраняем модель в Pickle
    #   with open(f"{run_name}_model.pkl", "wb") as f:
      #      pickle.dump(aml.leader, f)

        print(f"Результаты сохранены: {run_name}_metrics.json, {run_name}_leaderboard.csv, {run_name}_model.pkl, {run_name}_mojo.zip")
        print(results)

    return aml

In [36]:
# 3. Запуск AutoML с разными наборами признаков

# Таргет в формате H2OFrame
y = X_numerical_filtered[[target]]

# Запуск с RDKit-дескрипторами
aml_rdkit = run_h2o_automl(rdkit_descriptors, y, "RDKit Descriptors", saving=True)

# Запуск с эмбеддингами Blomap
blomap_df = pd.DataFrame(blomap_embeddings_filtered, index=filtered_indices)
aml_blomap = run_h2o_automl(blomap_df, y, "Blomap Embeddings")

# Запуск с эмбеддингами Fingerprints
fingerprints_df = pd.DataFrame(fingerprints_embeddings_filtered, index=filtered_indices)
aml_fingerprints = run_h2o_automl(fingerprints_df, y, "Fingerprints Embeddings")

# Запуск с эмбеддингами ProtBERT
protbert_df = pd.DataFrame(protbert_embeddings_filtered, index=filtered_indices)
aml_protbert = run_h2o_automl(protbert_df, y, "ProtBERT Embeddings")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |██
17:03:37.799: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 140.0.

█████████████████████████████████████████████████████████████| (done) 100%

=== RDKit Descriptors ===
model_id                                                     rmse          mse      mae      rmsle    mean_residual_deviance
GBM_4_AutoML_16_20250326_170336                           5986.24  3.58351e+07  3329.65  nan                     3.58351e+07
GBM_grid_1_AutoML_16_20250326_170336_model_1              6062.78  3.67572e+07  3391.63    3.34625               3.67572e+07
StackedEnsemble_AllModels_2_AutoML_16_20250326_170336     6090.02  3.70883e+07  3399.4     3.23768               3.70883e+07
StackedEnsemble_BestOfFamily_2_AutoML_16_20250326_170336  6124.62  3.75109e+07  3398.83  nan                     3.75109e+07
Sta

/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
17:03:49.742: _train param, Dropping bad and constant columns: [1220, 1217, 1216, 1215, 1214, 1213, 1212, 1211, 1210, 1219, 1218, 1231, 1230, 1228, 1227, 1226, 1225, 4, 1224, 1223, 1222, 1221, 800, 9, 801, 802, 803, 804, 805, 806, 807, 1229, 808, 809, 1000, 1242, 1241, 1240, 1239, 1238, 1237, 1236, 1235, 1234, 1233, 810, 1232, 811, 812, 813, 814, 815, 816, 817, 818, 819, 1011, 1253, 1010, 1252, 1251, 1250, 1008, 1007, 1249, 1006, 1248, 1005, 1247, 1004, 1246, 1003, 1245, 820, 1002, 1244, 821, 1001, 1243, 822, 823, 824, 825, 826, 827, 828, 829, 1009, 1022, 1264, 1021, 1263, 1020, 1262, 1261, 1260, 1019, 1018, 1017, 1259, 1016, 1258, 1015, 1257, 830, 1014, 1256, 831, 1013, 1255, 832, 1012, 1254, 833, 834, 835, 836, 837, 838, 839, 1033, 1275, 1032, 1274, 1031, 1273, 1030, 1272, 1271, 1270, 1029, 1028, 1027, 1269, 840, 1026, 1268, 841, 1025, 1267, 600, 842, 1024, 1266, 601, 843

In [ ]:
target_column = 'raw_efficiency'

# Разделение данных на тренировочные и тестовые
X_train, X_test = train_test_split(X_numerical, test_size=0.2, random_state=42)

In [ ]:
# Фильтрация данных по условию 'Mean Fluorescence intensity'
filtered_mask = X_numerical['uptake_type'] == 'Mean Fluorescence intensity'
X_filtered = X_numerical[filtered_mask].copy()

# Подготовка массивов эмбеддингов с фильтрацией
blomap_filtered = blomap_embeddings[filtered_mask]
fingerprints_filtered = fingerprints_embeddings[filtered_mask]
protbert_filtered = protbert_embeddings[filtered_mask]

In [ ]:
def filter_data(df, arrays, cell_line_array=None, is_y=False):
    """
    Фильтрует данные по маске из столбца 'uptake_type' датафрейма df.

    Параметры:
    - df: pandas.DataFrame с исходными данными
    - arrays: список массивов для фильтрации и объединения (например, [X_numerical, blomap_pca])
    - cell_line_array: дополнительный массив (X_cell_line), который может отсутствовать
    - is_y: флаг, что обрабатываем целевую переменную y

    Возвращает:
    - Отфильтрованный массив или датафрейм
    """
    # Создаем булеву маску
    mask = df['uptake_type'].isin(['Mean Fluorescence intensity', 'Fluorescence intensity'])

    if is_y:
        # Для y просто применяем маску к столбцу
        return df['raw_efficiency'].values[mask]
    else:
        # Фильтруем все переданные массивы
        filtered_arrays = [arr[mask] for arr in arrays]

        # Объединяем массивы
        filtered_X = np.hstack(filtered_arrays)

        # Добавляем cell_line, если он не пуст
        if cell_line_array is not None and not cell_line_array.empty:
            filtered_cell_line = cell_line_array.values[mask]
            filtered_X = np.hstack([filtered_X, filtered_cell_line])

        return filtered_X

In [ ]:
arrays_xgb = [
    X_numerical.values,
    blomap_embeddings,          # Из вашего исходного кода
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_xgb = filter_data(
    df=df,
    arrays=arrays_xgb,
    cell_line_array=X_cell_line  # Передаем X_cell_line, если он не пуст
)

# Для X_lgbm:
# Список массивов для X_lgbm (здесь другой blomap)
arrays_lgbm = [
    X_numerical.values,
    blomap_embeddings,    # Разный признак относительно X_xgb
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_lgbm = filter_data(
    df=df,
    arrays=arrays_lgbm,
    cell_line_array=X_cell_line
)

# Целевая переменная:
y = df["raw_efficiency"].values

y = filter_data(
    df=df,
    arrays=[],  # Для y не нужны дополнительные массивы
    cell_line_array=None,
    is_y=True   # Указываем, что обрабатываем целевую переменную
)

In [ ]:
# Apply PCA to Blomap for XGBoost
pca_blomap = PCA(n_components=10, random_state=42)
blomap_pca = pca_blomap.fit_transform(blomap_embeddings)




# Prepare feature matrices
X_xgb = np.hstack([X_numerical.values, blomap_pca, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_xgb = np.hstack([X_xgb, X_cell_line.values])

X_lgbm = np.hstack([X_numerical.values, blomap_embeddings, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_lgbm = np.hstack([X_lgbm, X_cell_line.values])

y = df["raw_efficiency"].values

# Пример создания исходных переменных:
# (предполагается, что переменные X_numerical, blomap_pca, fingerprints_embeddings,
#  protbert_embeddings, blomap_embeddings, X_cell_line и df уже определены)

# Для X_xgb:
# Список массивов для X_xgb
arrays_xgb = [
    X_numerical.values,
    blomap_pca,          # Из вашего исходного кода
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_xgb = filter_data(
    df=df,
    arrays=arrays_xgb,
    cell_line_array=X_cell_line  # Передаем X_cell_line, если он не пуст
)

# Для X_lgbm:
# Список массивов для X_lgbm (здесь другой blomap)
arrays_lgbm = [
    X_numerical.values,
    blomap_embeddings,    # Разный признак относительно X_xgb
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_lgbm = filter_data(
    df=df,
    arrays=arrays_lgbm,
    cell_line_array=X_cell_line
)

# Целевая переменная:
y = df["raw_efficiency"].values

y = filter_data(
    df=df,
    arrays=[],  # Для y не нужны дополнительные массивы
    cell_line_array=None,
    is_y=True   # Указываем, что обрабатываем целевую переменную
)


valid_idx = ~np.isnan(y)
X_xgb, X_lgbm, y = X_xgb[valid_idx], X_lgbm[valid_idx], y[valid_idx]

# Handle missing values
imputer = SimpleImputer(strategy="mean")
X_xgb, X_lgbm = imputer.fit_transform(X_xgb), imputer.fit_transform(X_lgbm)

# Log-transform target variable
y = np.log1p(y)